In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import fitz

In [9]:
sentence_transformer_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

c:\Users\saksh\OneDrive\Desktop\resume-4\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

jd_paths = ["jd_web_developer.txt", "jd_web_developer.txt", "jd_web_developer.txt", "jd_web_developer.txt", "Job Description.txt"]
# change resume_paths according the resume files uploaded
resume_paths = ["Hardik_Hans_resume_3.pdf", "Hardik_Hans_resume_4.pdf", "Hardik_Hans_resume_5.pdf", "dipankar_resume.pdf","aryancv1.pdf"]

jds_embeddings = [sentence_transformer_model.encode(extract_text_from_pdf(path)) for path in jd_paths]
resumes_embeddings = [sentence_transformer_model.encode(extract_text_from_pdf(path)) for path in resume_paths]

In [11]:
jds_embeddings_np = np.array(jds_embeddings)
resumes_embeddings_np = np.array(resumes_embeddings)

print("Shape of jds_embeddings:", jds_embeddings_np.shape)
print("Shape of resumes_embeddings:", resumes_embeddings_np.shape)

jds_train, jds_val, resumes_train, resumes_val = train_test_split(jds_embeddings_np, resumes_embeddings_np, test_size=0.4, random_state=42)

jd_embeddings_train = jds_train
resume_embeddings_train = resumes_train

Shape of jds_embeddings: (5, 384)
Shape of resumes_embeddings: (5, 384)


In [12]:
print("Shape of jd_embeddings_train:", jd_embeddings_train.shape)
print("Shape of resume_embeddings_train:", resume_embeddings_train.shape)

a, b = jd_embeddings_train.shape
print(a, b)

jd_embeddings_train = np.expand_dims(jd_embeddings_train, axis=1)

Shape of jd_embeddings_train: (3, 384)
Shape of resume_embeddings_train: (3, 384)
3 384


In [13]:
model = models.Sequential([
    layers.LSTM(128, input_shape=(1, b)),
    layers.Dense(64, activation='relu'),
    layers.Dense(resume_embeddings_train.shape[1], activation='sigmoid')  
])

model.compile(optimizer='adam', loss='mse') 

model.fit(x=jd_embeddings_train, y=resume_embeddings_train, epochs=10, batch_size=32)

Epoch 1/10


c:\Users\saksh\OneDrive\Desktop\resume-4\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.2529
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.2524
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2520
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2515
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2509
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2503
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2495
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2486
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2475
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.2462


In [14]:
def rank_resumes(job_description, resumes):
    jd_embedding = job_description
    resume_embeddings = resumes
    similarities = cosine_similarity(jd_embedding, resume_embeddings)
    ranked_resumes_indexes = np.argsort(similarities)[::-1]
    return ranked_resumes_indexes, similarities

ranked_resumes_indexes, similarities = rank_resumes(jds_val, resumes_val)

print("Ranked Resumes:")
print(similarities)

Ranked Resumes:
[[0.4694916  0.07508232]
 [0.15682328 0.27176112]]
